# 動態網頁爬蟲 - 使用 Selenium

* 了解 Selenium 用於動態網頁爬蟲的原理
* 能夠使用 Selenium 撰寫動態網頁爬蟲

## 作業目標


* 根據範例使用 selenuim 取出 高雄市小港區 2019 年 空氣品質 資料，並用一個適合的資料結構儲存
* Hint: 將所有資料存在一個變數當中



In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

browser = webdriver.Firefox(executable_path='/usr/bin/geckodriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_visible_text('高雄市-小港')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2019')

browser.find_element_by_id('ctl05_btnQuery').click()

![](https://imgur.com/LwGGT8c.jpg)

In [2]:
# 取得資料（等瀏覽器資料出現後才執行）
html_source = browser.page_source
# html_source

In [3]:
soup = BeautifulSoup(html_source)
table = soup.find('table', class_='TABLE_G')
# table

In [4]:
aq_dict = []
tr_list = table.find_all('tr')
for tr in tr_list:
    td_list = tr.find_all('td')
    if len(td_list) > 3:
        aq_dict.append({'監測項目': td_list[0].text, '單位': td_list[1].text, '監測日期': [], '監測值': []})
        aq_dict[-1]['監測日期'].append(td_list[2].text)
        aq_dict[-1]['監測值'].append(td_list[3].text)
    elif len(td_list) == 3:
        aq_dict[-1]['監測日期'].append(td_list[0].text)
        aq_dict[-1]['監測值'].append(td_list[1].text)
# aq_dict

In [71]:
import pandas as pd

df = pd.DataFrame(aq_dict[0]['監測日期'], columns={'監測日期'})
for item in aq_dict:
    df_ = pd.DataFrame(item['監測值'], columns={'%s (%s)' % (item['監測項目'], item['單位'])})
    df = pd.concat([df, df_], axis=1, sort=False)
df

,監測日期,SO2 (ppb),CO (ppm),O3 (ppb),PM10 (μg/m3),NOx (ppb),NO (ppb),NO2 (ppb),THC (ppm),NMHC (ppm),CH4 (ppm)
0,2019/01,4.10,0.67,24.70,78,38.01,7.95,30.06,2.27,0.27,2
1,2019/02,3.30,0.48,29.40,57,25.62,4.76,20.86,2.11,0.17,1.94
2,2019/03,3.90,0.50,29.20,52,27.84,5.41,22.43,2.12,0.18,1.94
3,2019/04,3.40,0.39,30.40,42,21.83,4.16,17.68,2.02,0.14,1.88
4,2019/05,3.30,0.38,28.60,35,22.55,4.58,17.98,2.02,0.16,1.86
5,2019/06,4.10,0.24,16.30,20,17.15,5.08,12.08,1.92,0.13,1.79
6,2019/07,4,0.28,15.50,24,17.71,5.16,12.54,1.93,0.14,1.80
7,2019/08,4.60,0.33,19.70,27,19.66,5.17,14.49,1.97,0.15,1.82
8,2019/09,3.50,0.32,35.70,35,18.66,3.63,15.03,1.98,0.13,1.86
9,2019/10,3.20,0.36,44,50,19.50,3.50,15.99,2.05,0.13,1.92
